## Hall Effect Heating and Cooling Simulation

## To-Do
- Get schematics of hall bar. Materials, dimensions, ect
- Read about heat transfer functions
- Investigate the maximum heat that can be given from the heater. (In terms of safe currents to run through to get an idea of what values to use).
- Make reposototory work

### Considerations

- Thermal diffusity constant of materials

Made of copper/brass (i think one of those) and outside made of stainless steel (maybe).

- Model heat loss to the environment

Radiation, convection, conduction (not this one for air?). Figure out which one dominates. Is it important to model both? Radiation and convection from room? 

- Consider the fact we are cooling from the outside of the hall bar but heating from the inside.

- Heating

Find the relationship between current given to the resistor in the hall bar and how much heat is produced. Can that just be measure the power going in and then approximate that all that power is dissipated through the resistor.

- Overall measure how much energy is flowing in or out.

- Use finite element analysis

- Consider how we actually put the coolant in. Can only put so much in at one time. Try and make it efficient as well so it doesn't all evaporate (Can be a point for improving the experiment).

## Planning

### Heat transfer methods - Conduction

Read wikipedia page do we use the heat transfer function from numerical methods?

### Heat transfer methods - Radiation

Read wikipedia page do we use the heat transfer function from numerical methods?

### Heat transfer methods - Convection

Read wikipedia page do we use the heat transfer function from numerical methods?

### Implementation
- Bulid a simple simulation of a rectange with a heat sink on the outside and heat source on the inside. Make it arbitrary dimensions for now.
- Try and add a function to say when the temperature of the whole thing is within a certain range and what the difference between the highest and lowest temperatures are.
- Is there a way to have 2 seperate diffusity constants? Is this necessary.
- How does heat loss/gain to/from the environment work/get modelled.
- Calculate the stability constants of the differential equations

### Testing
- Is there a way to test the heat loss and gain equations are working properly?
- Track conservation of energy (This will need to account for energy going in and out of the system. But is that all consistent with the values inside?)